In [ ]:
# reload modules automatically


#%reload_ext autoreload

#%reload_ext storage.create_data_warehouse

In [ ]:
# Importing necessary libraries and modules 

import subprocess

try:

    import os
    import boto3
    import time
    from memory_profiler import profile

    import psycopg2




except ImportError as error:
    print(f"Installation of the required dependencies necessary! {error}")

    subprocess.check_call(["pip", "install", "boto3"])


    print(f"Successful installation of the required dependencies necessary")


import warnings
warnings.filterwarnings('ignore')


# custom imports

## data lakes aws
from storage import (
    configure_aws_session,
    create_aws_data_lake,
    create_data_warehouse,
    create_db_tables,
    load_data_db_tables,
    )



In [ ]:
# get file paths

current_dir = os.path.dirname(os.path.abspath('__file__'))

print(f''' current directory :
        {current_dir} ''')

root_path = os.path.normpath(os.path.join(current_dir, '../data/raw/'))

client_path = os.path.normpath(os.path.join(root_path, 'client_data.csv'))
invoice_path = os.path.normpath(os.path.join(root_path, 'invoice_data.csv'))


data_files_path = [client_path, invoice_path]


In [ ]:
# create session

session = configure_aws_session()



In [ ]:
# create aws data lake: s3 bucket to store fraud detection data files

create_aws_data_lake(session, data_files_path, bucket_name= 'fraud-detection-datasets')



In [ ]:
# create data warehouse: creating postgresql rds instance in aws

rds_db_config, response = create_data_warehouse()


In [ ]:
# create tables in the data warehouse using posgres psycopg2 driver

sql_create_client_table = '''
        CREATE TABLE IF NOT EXISTS client_data (
            district  TEXT,
            client_id INT,
            client_catg  TEXT,
            region TEXT,
            creation_date DATE,
            target DECIMAL,
        );
        '''



sql_create_invoice_table = '''
        CREATE TABLE IF NOT EXISTS invoice_data (
            client_id INT,
            invoice_date DATE,
            tarif_type TEXT,
            counter_number INT,
            counter_statue TEXT,
            counter_code TEXT,
            reading_remarque TEXT,
            counter_coefficient DECIMAL,
            consommation_level_1 DECIMAL,
            consommation_level_2 DECIMAL,
            consommation_level_3 DECIMAL,
            consommation_level_4 DECIMAL,
            old_index DECIMAL,
            new_index DECIMAL,
            months_number INT,
            counter_type TEXT
        );
        '''


sql_queries = [sql_create_client_table, sql_create_invoice_table]

create_db_tables(rds_db_config, sql_queries)


In [ ]:
# get data from bucket(data lake) into data warehouse tables in postgres

load_data_db_tables(session, rds_db_config, data_files_path, bucket_name= 'fraud_detection_datasets')

